In [1]:
from expert2 import *
import wrinkle2
import util
import cv2
import matplotlib.pyplot as plt

class Dagger:
    frame_id = 0
    def __init__(self):
        import time
        path = "./"+time.strftime("%m%d-%H%M")
        if os.path.exists(path):
            shutil.rmtree(path)
        os.mkdir(path)

        self.tt_expert_pos = np.array([])
        self.tt_pred_pos = np.array([])
        self.tt_handles = np.array([])
        self.tt_feat = np.array([])
        self.tt_log = np.array([])
        self.path = path
        pass

    def update(self,handles, pred_pos, expert_pos, feat, log):
        handles = np.array(handles).ravel();
        expert_pos = np.array(expert_pos).ravel()
        pred_pos = np.array(pred_pos).ravel()
        

        self.tt_pred_pos = np.vstack((self.tt_pred_pos,pred_pos)) if self.tt_pred_pos.size else pred_pos
        self.tt_feat = np.vstack((self.tt_feat,feat)) if self.tt_feat.size else feat
        self.tt_handles = np.vstack((self.tt_handles, handles)) if self.tt_handles.size else handles
        self.tt_expert_pos = np.vstack((self.tt_expert_pos, expert_pos)) if self.tt_expert_pos.size else expert_pos
        self.tt_log = np.vstack((self.tt_log, this_log)) if self.tt_log.size else this_log
        
        np.savez(self.path+'/data',tt_expert_pos = self.tt_expert_pos,
                 tt_pred_pos = self.tt_pred_pos, tt_feat = self.tt_feat, 
                 tt_handles = self.tt_handles, tt_log = self.tt_log)
        pass
    
    def frame_path(self):
        self.frame_id = self.frame_id + 1
        return self.path+"/%04i"%(self.frame_id-1)
    
    def saved_frame_path(self):
        return self.path+"/%04i"%(self.frame_id-1)+'.png'
    
    
    
def random_hands(x):
    hands=np.zeros((2,3))
    # pick two random positions for hands, which have at most x (won't break the cloth)
    while True:
        # random from [0,0,0] [x,0,0], which is the first two handles
        hands[0] = [random.uniform(-x/4,x/4),random.uniform(-x/4,x/4),random.uniform(-x/4,x/4)]
        hands[1] = [random.uniform(-x/4,x/4)+x,random.uniform(-x/4,x/4),random.uniform(-x/4,x/4)]
        if np.linalg.norm(np.subtract(hands[0],hands[1])) < x:
            break
    
    return hands
    
    
remesh_fag = False
render_image = True
render_depth = False
MOVEMENT_THRESH = 1e-5
BETA = 0.95
cloth_x = 1.0 #width of the cloth 
cloth_y = 2.0 #length of the cloth
delta = 0.01 # controller gain, multiplier
nr_frame = 100 # number of frames each setup consists
nr_pass = 30 # number of different passes (setups)

expert = arcsim_expert()
expert.create_sheet(1,5,2,10,"./python_sheet",[0,0,-9.81],[0,0,0],0,0,remesh_fag)
expert.setup("./python_sheet/sheet.json")
expert_controller = expert.expert_flat #expert type

# initialize the positions
handles=[[0,0,0],[cloth_x,0,0],[0,cloth_y,0],[cloth_x,cloth_y,0]]
# frame_id
dagger = Dagger()

for p in range(nr_pass):
    hands = random_hands(x=cloth_x)
    pos = np.zeros((2,3))    
    BETA_p = BETA**p
    # start each run
    for i in range(nr_frame):
        # handles
        handles_prev = handles
        expert.advance()
        handles = expert.apply_hand(handles, hands, delta)
        handles = expert.apply_expert(handles, pos, delta)
        expert.set_handle(handles)
        # save images
        expert.save_frame(dagger.frame_path(), render_image, render_depth)
        print dagger.saved_frame_path()
        # get the visual feature
        im = cv2.imread(dagger.saved_frame_path())
        feat = wrinkle2.xhist(im)
        expert_pos = np.array(expert_controller(handles,cloth_x,cloth_y))
        if p>0:
            pred_pos = rt.feat_to_act(feat=feat.reshape(1,-1))
        else:
            pred_pos = np.zeros(expert_pos.shape)
        expert.advance()
        
        # apply the robot controllers positions
        if random.random()>BETA_p:
            use_pred = 1
            pos = pred_pos
        else:
            use_pred = 0
            pos = expert_pos        

        this_log = np.array([p,i,use_pred]);
        dagger.update(log=this_log, handles=handles_prev, pred_pos=pred_pos, expert_pos=expert_pos, feat=feat)
        
        # determine whether to terminate the run
        d = np.array(handles_prev).ravel() - np.array(handles).ravel()
        d = sum(d*d)
        if d<MOVEMENT_THRESH:
            break;
        
    # begin training
    from rfdic import random_trees
    num_class = 10
    rt = random_trees(tt_handles=dagger.tt_handles, 
                      tt_pos=dagger.tt_expert_pos, 
                      tt_feat=dagger.tt_feat, 
                      num_class=num_class, 
                      num_trees=10)

./1116-2001/0000.png


/home/rmqlife/anaconda2/lib/python2.7/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


./1116-2001/0001.png
./1116-2001/0002.png
./1116-2001/0003.png
./1116-2001/0004.png
./1116-2001/0005.png
./1116-2001/0006.png
./1116-2001/0007.png
./1116-2001/0008.png
./1116-2001/0009.png
./1116-2001/0010.png
./1116-2001/0011.png
./1116-2001/0012.png
./1116-2001/0013.png
./1116-2001/0014.png
./1116-2001/0015.png
./1116-2001/0016.png
./1116-2001/0017.png
./1116-2001/0018.png
./1116-2001/0019.png
./1116-2001/0020.png
./1116-2001/0021.png
./1116-2001/0022.png
./1116-2001/0023.png
./1116-2001/0024.png
./1116-2001/0025.png
./1116-2001/0026.png
./1116-2001/0027.png
./1116-2001/0028.png
./1116-2001/0029.png
./1116-2001/0030.png
./1116-2001/0031.png
./1116-2001/0032.png
./1116-2001/0033.png
./1116-2001/0034.png
./1116-2001/0035.png
./1116-2001/0036.png
./1116-2001/0037.png
./1116-2001/0038.png
./1116-2001/0039.png
./1116-2001/0040.png
./1116-2001/0041.png
./1116-2001/0042.png
./1116-2001/0043.png
./1116-2001/0044.png
./1116-2001/0045.png
./1116-2001/0046.png
./1116-2001/0047.png
./1116-2001/0

IOError: POVRay rendering failed with the following error: povray: cannot open the user configuration file /home/rmqlife/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.unofficial (g++ 5.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Martin Pitt <martin.pitt@ubuntu.com> for Debian <www.debian.org>
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet    

In [2]:
print dagger.tt_log

[[ 0  0  0]
 [ 0  1  0]
 [ 0  2  0]
 [ 0  3  0]
 [ 0  4  0]
 [ 0  5  0]
 [ 0  6  0]
 [ 0  7  0]
 [ 0  8  0]
 [ 0  9  0]
 [ 0 10  0]
 [ 0 11  0]
 [ 0 12  0]
 [ 0 13  0]
 [ 0 14  0]
 [ 0 15  0]
 [ 0 16  0]
 [ 0 17  0]
 [ 0 18  0]
 [ 0 19  0]
 [ 0 20  0]
 [ 0 21  0]
 [ 0 22  0]
 [ 0 23  0]
 [ 0 24  0]
 [ 0 25  0]
 [ 0 26  0]
 [ 0 27  0]
 [ 0 28  0]
 [ 0 29  0]
 [ 0 30  0]
 [ 0 31  0]
 [ 0 32  0]
 [ 0 33  0]
 [ 0 34  0]
 [ 0 35  0]
 [ 0 36  0]
 [ 0 37  0]
 [ 0 38  0]
 [ 0 39  0]
 [ 0 40  0]
 [ 0 41  0]
 [ 0 42  0]
 [ 0 43  0]
 [ 0 44  0]
 [ 0 45  0]
 [ 0 46  0]
 [ 0 47  0]
 [ 0 48  0]
 [ 0 49  0]
 [ 0 50  0]
 [ 0 51  0]
 [ 0 52  0]
 [ 0 53  0]
 [ 0 54  0]
 [ 0 55  0]]
